Extract dataset from zip file

In [9]:
import zipfile

data_zip = zipfile.ZipFile("./data.zip")
data_zip.extractall()
data_zip.close()

Collect class (person) names from dataset folders

In [15]:
import os
classes = []

for id in os.listdir("./data"):
    classes.append(id)
print(classes)
print(len(classes))

['22-47884-2', '22-49862-3', '22-48569-3', '22-48133-2 ', '22-49843-3', '22-49037-3', '22-46258-1', '22-46139-1', '22-49451-3', '23-50346-1', '23-51308-1', '22-48205-2', '22-46590-1', '22-49355-3', '22-49370-3', '23-51127-1', '22-48005-2', '22-49783-3', '23-50277-1', '22-46342-1', '21-45902-3', '22-48682-3', '22-49824-3', '22-48725-3', '22-49575-3', '23-50066-1', '22-46138-1', '22-48582-3', '22-48915-3', '22-47892-2', '22-47898-2', '22-48841-3', '22-46536-1', '22-48021-2', '22-48023-2_', '22-49196-3', '22-49331-3', '22-48666-3', '22-49791-3', '22-48091-2', '22-49068-3', '22-47968-2', '22-49538-3', '22-48833-3', '23-50254-1', '22-47542-2', '22-49507-3', '22-49852-3', '22-46983-1', '22-47813-2', '22-49643-3', '22-49338-3', '22-46887-1']
53


In [ ]:
import random
import shutil

# Define source and destination directories
source_dir = "./data"
dest_dir = "./data_split"

# Define dataset split ratios
SPLIT = {
    "train": 0.7,
    "validation": 0.15,
    "test": 0.15
}

random.seed(42)

# Split images into train, validation, and test sets
for cls in classes:
    src_path = os.path.join(source_dir, cls);
    images = os.listdir(src_path)
    random.shuffle(images)

    total = len(images)
    train_end = int(total * SPLIT["train"])
    val_end = train_end + int(total * SPLIT["validation"])

    split_data = {
        "train": images[:train_end],
        "validation": images[train_end:val_end],
        "test": images[val_end:]
    }

    for split, files in split_data.items():
        split_dir = os.path.join(dest_dir, split, cls)
        os.makedirs(split_dir, exist_ok=True)

        for file in files:
            shutil.copy(
                os.path.join(src_path, file),
                os.path.join(split_dir, file)
            )
            

Define dataset directories

In [18]:
train_dir = "./data_split/train"
val_dir = "./data_split/validation"
test_dir = "./data_split/test"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Normalize image pixel values
train_datagen = ImageDataGenerator(rescale=1/255.0)
val_datagen = ImageDataGenerator(rescale=1/255.0)
test_datagen = ImageDataGenerator(rescale=1/255.0)

# Load training images from directory
train_data = train_datagen.flow_from_directory(train_dir,
                                             target_size=(224,224),
                                             batch_size=16,
                                             class_mode="categorical")
# Load validation images from directory
val_data = val_datagen.flow_from_directory(val_dir,
                                             target_size=(224,224),
                                             batch_size=16,
                                             class_mode="categorical"
                                            )
# Load test images from directory
test_data = test_datagen.flow_from_directory(test_dir,
                                            target_size=(224,224),
                                            batch_size=16,
                                            class_mode="categorical")

Found 483 images belonging to 53 classes.
Found 103 images belonging to 53 classes.
Found 110 images belonging to 53 classes.


Define CNN model architecture

In [29]:
import tensorflow as tf

model_1 = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(224,224,3),),
    tf.keras.layers.Conv2D(filters=10,
                          kernel_size=(3,3),
                          ),

    tf.keras.layers.Activation(activation="relu"),
    tf.keras.layers.Conv2D(filters=10,
                           kernel_size=(3,3),
                           activation="relu",
                          ),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=10,
                           kernel_size=(3,3),
                           activation="relu",
                          ),
    tf.keras.layers.Conv2D(filters=10,
                           kernel_size=(3,3),
                           activation="relu",
                          ),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(len(classes), activation="softmax")
    
])

Compile the model

In [30]:
model_1.compile(loss="categorical_crossentropy",
               optimizer=tf.keras.optimizers.Adam(),
               metrics=["accuracy"])

Train the model

In [31]:
history_1 = model_1.fit(train_data,
                       epochs=10,
                       steps_per_epoch=len(train_data),
                       validation_data=val_data,
                       validation_steps=len(val_data))

Epoch 1/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 21s 678ms/step - accuracy: 0.0973 - loss: 3.4931 - val_accuracy: 0.4175 - val_loss: 2.7236
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 658ms/step - accuracy: 0.6025 - loss: 1.7921 - val_accuracy: 0.6019 - val_loss: 1.6203
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 20s 658ms/step - accuracy: 0.8116 - loss: 0.6885 - val_accuracy: 0.7864 - val_loss: 0.8798
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 21s 664ms/step - accuracy: 0.9441 - loss: 0.2214 - val_accuracy: 0.8058 - val_loss: 0.9304
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 21s 682ms/step - accuracy: 0.9793 - loss: 0.0661 - val_accuracy: 0.7670 - val_loss: 1.1297
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 21s 667ms/step - accuracy: 0.9917 - loss: 0.0266 - val_accuracy: 0.8155 - val_loss: 0.9607
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 21s 669ms/step - accuracy: 0.9979 - loss: 0.0196 - val_accuracy: 0.7961 - val_loss: 0.8491
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 21s 670ms/step - accuracy: 1.0000 - loss: 0.0030 - val_accu

Evaluate model performance on test data

In [32]:
model_1.evaluate(test_data)

7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 589ms/step - accuracy: 0.8273 - loss: 1.1731


[1.1731129884719849, 0.8272727131843567]

Save trained model

In [33]:
model_1.save("attendence.keras")

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import os
import pathlib


model = load_model("attendence.keras")


data_dir = pathlib.Path("./data_split/train")

class_names = np.array([
    item.name
    for item in data_dir.glob("*")
    if item.is_dir() and not item.name.startswith(".")
])

print(class_names)


# Face detector
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)


# Prediction function
def predict_face(face):
    face_resized = cv2.resize(face, (224, 224))  
    face_resized = face_resized / 255.0
    face_expanded = np.expand_dims(face_resized, axis=0)

    preds = model.predict(face_expanded)[0]

    # Highest scoring class
    class_id = np.argmax(preds)
    confidence = preds[class_id] * 100  # Convert to %

    return class_names[class_id], confidence


# Webcam loop
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]

        name, confidence = predict_face(face_roi)

        if confidence >= 95:
            text = f"{name} ({confidence:.2f}%)"
            color = (0, 255, 0)
        else:
            text = "Unknown"
            color = (0, 0, 255)

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, text, (x, y-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

    cv2.imshow("Attendance System", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


['22-47884-2' '22-49862-3' '22-48569-3' '22-48133-2 ' '22-49843-3'
 '22-49037-3' '22-46258-1' '22-46139-1' '22-49451-3' '23-50346-1'
 '23-51308-1' '22-48205-2' '22-46590-1' '22-49355-3' '22-49370-3'
 '23-51127-1' '22-48005-2' '22-49783-3' '23-50277-1' '22-46342-1'
 '21-45902-3' '22-48682-3' '22-49824-3' '22-48725-3' '22-49575-3'
 '23-50066-1' '22-46138-1' '22-48582-3' '22-48915-3' '22-47892-2'
 '22-47898-2' '22-48841-3' '22-46536-1' '22-48021-2' '22-48023-2_'
 '22-49196-3' '22-49331-3' '22-48666-3' '22-49791-3' '22-48091-2'
 '22-49068-3' '22-47968-2' '22-49538-3' '22-48833-3' '23-50254-1'
 '22-47542-2' '22-49507-3' '22-49852-3' '22-46983-1' '22-47813-2'
 '22-49643-3' '22-49338-3' '22-46887-1']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12m